<a href="https://colab.research.google.com/github/gupta24789/sentiment-analysis/blob/main/sentiment_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## CNN

  Traditionally, CNNs are used to analyse images and are made up of one or more convolutional layers, followed by one or more linear layers. The convolutional layers use filters (also called kernels or receptive fields) which scan across an image and produce a processed version of the image. This processed version of the image can be fed into another convolutional layer or a linear layer. Each filter has a shape, e.g. a 3x3 filter covers a 3 pixel wide and 3 pixel high area of the image, and each element of the filter has a weight associated with it, the 3x3 filter would have 9 weights. In traditional image processing these weights were specified by hand by engineers, however the main advantage of the convolutional layers in neural networks is that these weights are learned via backpropagation.

  The intuitive idea behind learning the weights is that your convolutional layers act like feature extractors, extracting parts of the image that are most important for your CNN's goal, e.g. if using a CNN to detect faces in an image, the CNN may be looking for features such as the existance of a nose, mouth or a pair of eyes in the image.

  So why use CNNs on text? In the same way that a 3x3 filter can look over a patch of an image, a 1x2 filter can look over a 2 sequential words in a piece of text, i.e. a bi-gram. In this CNN model we will instead use multiple filters of different sizes which will look at the bi-grams (a 1x2 filter), tri-grams (a 1x3 filter) and/or n-grams (a 1x n filter) within the text.

  The intuition here is that the appearance of certain bi-grams, tri-grams and n-grams within the review will be a good indication of the final sentiment.

In [1]:
!pip install -q pytorch-lightning

In [2]:
import pandas as pd
import numpy as np
import itertools
import warnings

import re
import string
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords

nltk.download('stopwords')

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.nn.utils.rnn import  pad_sequence
from torch.utils.data import Dataset, DataLoader

from torchmetrics import Accuracy
import pytorch_lightning as pl

warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Set Seed

In [3]:
## set seed
np.random.seed(121)
torch.manual_seed(121)
pl.seed_everything(121)

INFO:lightning_fabric.utilities.seed:Seed set to 121


121

## Utilities

In [4]:
def process_tweet(tweet):
    """Process tweet function.
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet

    """
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    tweet = re.sub(r'$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation):  # remove punctuation
            # tweets_clean.append(word)
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)

    return tweets_clean

## Load Data

In [5]:
train_df = pd.read_csv("https://raw.githubusercontent.com/gupta24789/sentiment-analysis/main/data/train.csv")
val_df = pd.read_csv("https://raw.githubusercontent.com/gupta24789/sentiment-analysis/main/data/val.csv")

train_df.processed_tweet = train_df.processed_tweet.fillna('[]').apply(lambda x: eval(x) if x is not None else [])
val_df.processed_tweet = val_df.processed_tweet.fillna('[]').apply(lambda x: eval(x) if x is not None else [])

## remove blank
train_df = train_df[train_df.processed_tweet.str.len()!=0]
val_df = val_df[val_df.processed_tweet.str.len()!=0]

train_df = train_df.dropna()
val_df = val_df.dropna()

## reset index
train_df = train_df.reset_index(drop = True)
val_df = val_df.reset_index(drop = True)

In [6]:
train_df.label.value_counts()

0.0    3999
1.0    3987
Name: label, dtype: int64

In [7]:
val_df.label.value_counts()

0    1000
1     999
Name: label, dtype: int64

In [8]:
train_df.head(3)

,raw_tweet,processed_tweet,label
0,Want to say a huge thanks to @WarriorAssaultS ...,"[want, say, huge, thank, ff, thank, support, :)]",1.0
1,@jaynehh_ you just need a job and get a letter...,"[need, job, get, letter, work, place, say, wor...",1.0
2,"@knhillrocks HA yes, make it quick tho :D","[ha, ye, make, quick, tho, :d]",1.0


## Build Vocab

In [9]:
special_words = ['__PAD__','__UNK__','</e>']
unique_words = list(set(itertools.chain.from_iterable(train_df.processed_tweet.tolist())))
vocab = special_words + unique_words
vocab = {w:i for i,w in enumerate(vocab)}
print(f"Number of words in vocab : {len(vocab)}")

Number of words in vocab : 9092


## Convert tweet to number

In [10]:
def tweet_to_tensor(processed_tweet_list, unk_token = '__UNK__'):
  to_tensor_list = []
  unk_token_id = vocab[unk_token]

  for w in processed_tweet_list:
    to_tensor_list.append(vocab.get(w,unk_token_id))

  to_tensor = torch.tensor(to_tensor_list)
  return to_tensor

In [11]:
train_df['tensor_tweet'] = [tweet_to_tensor(tweet) for tweet in train_df.processed_tweet]
val_df['tensor_tweet'] = [tweet_to_tensor(tweet) for tweet in val_df.processed_tweet]

In [12]:
train_df.head(3)

,raw_tweet,processed_tweet,label,tensor_tweet
0,Want to say a huge thanks to @WarriorAssaultS ...,"[want, say, huge, thank, ff, thank, support, :)]",1.0,"[tensor(5944), tensor(2935), tensor(7489), ten..."
1,@jaynehh_ you just need a job and get a letter...,"[need, job, get, letter, work, place, say, wor...",1.0,"[tensor(2211), tensor(5247), tensor(958), tens..."
2,"@knhillrocks HA yes, make it quick tho :D","[ha, ye, make, quick, tho, :d]",1.0,"[tensor(2678), tensor(2308), tensor(3222), ten..."


In [13]:
train_data = train_df[['tensor_tweet','label']].reset_index(drop = True).to_dict("records")
val_data = val_df[['tensor_tweet','label']].reset_index(drop = True).to_dict("records")

## Data Loaders

In [14]:
def custom_collate(data):
  features = [d['tensor_tweet'] for d in data]
  labels = [d['label'] for d in data]
  padded_features = pad_sequence(features, batch_first=True, padding_value= vocab['__PAD__'])
  labels = torch.tensor(labels, dtype = torch.float32)
  batch = {"features": padded_features,"labels": labels}
  return batch

In [15]:
train_dl = DataLoader(train_data, batch_size = 2, collate_fn = custom_collate, shuffle = True)
example = next(iter(train_dl))
feature, label = example['features'], example['labels']


In [16]:
feature

tensor([[5947, 7646,   97, 1777,   23, 6505, 2359, 6113, 2866, 2597],
        [ 770, 4123, 6589,  576, 5724, 4617, 2935, 1095,  576,    0]])

In [17]:
label

tensor([0., 1.])

In [18]:
### dataloader
BATCH_SIZE = 64
train_dl = DataLoader(train_data, batch_size = BATCH_SIZE, collate_fn = custom_collate, shuffle = True)
val_dl = DataLoader(val_data, batch_size = BATCH_SIZE, collate_fn = custom_collate, shuffle = False)

## Build the Model

Now to build our model.

The first major hurdle is visualizing how CNNs are used for text. Images are typically 2 dimensional (we'll ignore the fact that there is a third "colour" dimension for now) whereas text is 1 dimensional. However, we know that the first step is converting the words into word embeddings. This is how we can visualize our words in 2 dimensions, each word along one axis and the elements of vectors aross the other dimension. Consider the 2 dimensional representation of the embedded sentence below:

![](https://github.com/gupta24789/sentiment-analysis/blob/main/imgaes/sentiment9.png?raw=1)

We can then use a filter that is **[n x emb_dim]**. This will cover $n$ sequential words entirely, as their width will be `emb_dim` dimensions. Consider the image below, with our word vectors are represented in green. Here we have 4 words with 5 dimensional embeddings, creating a [4x5] "image" tensor. A filter that covers two words at a time (i.e. bi-grams) will be **[2x5]** filter, shown in yellow, and each element of the filter with have a _weight_ associated with it. The output of this filter (shown in red) will be a single real number that is the weighted sum of all elements covered by the filter.

![](https://github.com/gupta24789/sentiment-analysis/blob/main/imgaes/sentiment12.png?raw=1)

The filter then moves "down" the image (or across the sentence) to cover the next bi-gram and another output (weighted sum) is calculated.

![](https://github.com/gupta24789/sentiment-analysis/blob/main/imgaes/sentiment13.png?raw=1)

Finally, the filter moves down again and the final output for this filter is calculated.

![](https://github.com/gupta24789/sentiment-analysis/blob/main/imgaes/sentiment14.png?raw=1)

In our case (and in the general case where the width of the filter equals the width of the "image"), our output will be a vector with number of elements equal to the height of the image (or lenth of the word) minus the height of the filter plus one, $4-2+1=3$ in this case.

This example showed how to calculate the output of one filter. Our model (and pretty much all CNNs) will have lots of these filters. The idea is that each filter will learn a different feature to extract. In the above example, we are hoping each of the **[2 x emb_dim]** filters will be looking for the occurence of different bi-grams.

In our model, we will also have different sizes of filters, heights of 3, 4 and 5, with 100 of each of them. The intuition is that we will be looking for the occurence of different tri-grams, 4-grams and 5-grams that are relevant for analysing sentiment of movie reviews.

The next step in our model is to use *pooling* (specifically *max pooling*) on the output of the convolutional layers implemented by the `F.avg_pool2d` function, however instead of taking the average over a dimension, we are taking the maximum value over a dimension. Below an example of taking the maximum value (0.9) from the output of the convolutional layer on the example sentence (not shown is the activation function applied to the output of the convolutions).

![](https://github.com/gupta24789/sentiment-analysis/blob/main/imgaes/sentiment15.png?raw=1)

The idea here is that the maximum value is the "most important" feature for determining the sentiment of the review, which corresponds to the "most important" n-gram within the review. How do we know what the "most important" n-gram is? Luckily, we don't have to! Through backpropagation, the weights of the filters are changed so that whenever certain n-grams that are highly indicative of the sentiment are seen, the output of the filter is a "high" value. This "high" value then passes through the max pooling layer if it is the maximum value in the output.

As our model has 100 filters of 3 different sizes, that means we have 300 different n-grams the model thinks are important. We concatenate these together into a single vector and pass them through a linear layer to predict the sentiment. We can think of the weights of this linear layer as "weighting up the evidence" from each of the 300 n-grams and making a final decision.

### Implementation Details

We implement the convolutional layers with `nn.Conv2d`. The `in_channels` argument is the number of "channels" in your image going into the convolutional layer. In actual images this is usually 3 (one channel for each of the red, blue and green channels), however when using text we only have a single channel, the text itself. The `out_channels` is the number of filters and the `kernel_size` is the size of the filters. Each of our `kernel_size`s is going to be **[n x emb_dim]** where $n$ is the size of the n-grams.

In PyTorch, RNNs want the input with the batch dimension second, whereas CNNs want the batch dimension first - we do not have to permute the data here as we have already set `batch_first = True`. We then pass the sentence through an embedding layer to get our embeddings. The second dimension of the input into a `nn.Conv2d` layer must be the channel dimension. As text technically does not have a channel dimension, we `unsqueeze` our tensor to create one. This matches with our `in_channels=1` in the initialization of our convolutional layers.

We then pass the tensors through the convolutional and pooling layers, using the `ReLU` activation function after the convolutional layers. Another nice feature of the pooling layers is that they handle sentences of different lengths. The size of the output of the convolutional layer is dependent on the size of the input to it, and different batches contain sentences of different lengths. Without the max pooling layer the input to our linear layer would depend on the size of the input sentence (not what we want). One option to rectify this would be to trim/pad all sentences to the same length, however with the max pooling layer we always know the input to the linear layer will be the total number of filters. **Note**: there an exception to this if your sentence(s) are shorter than the largest filter used. You will then have to pad your sentences to the length of the largest filter.

Finally, we perform dropout on the concatenated filter outputs and then pass them through a linear layer to make our predictions.

In [19]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim,
                 dropout, pad_idx):

        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)

        self.conv_0 = nn.Conv2d(in_channels = 1,
                                out_channels = n_filters,
                                kernel_size = (filter_sizes[0], embedding_dim))

        self.conv_1 = nn.Conv2d(in_channels = 1,
                                out_channels = n_filters,
                                kernel_size = (filter_sizes[1], embedding_dim))

        self.conv_2 = nn.Conv2d(in_channels = 1,
                                out_channels = n_filters,
                                kernel_size = (filter_sizes[2], embedding_dim))

        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, text):

        #text = [batch size, sent len]

        embedded = self.embedding(text)

        #embedded = [batch size, sent len, emb dim]

        embedded = embedded.unsqueeze(1)

        #embedded = [batch size, 1, sent len, emb dim]

        conved_0 = F.relu(self.conv_0(embedded).squeeze(3))
        conved_1 = F.relu(self.conv_1(embedded).squeeze(3))
        conved_2 = F.relu(self.conv_2(embedded).squeeze(3))

        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]

        pooled_0 = F.max_pool1d(conved_0, conved_0.shape[2]).squeeze(2)
        pooled_1 = F.max_pool1d(conved_1, conved_1.shape[2]).squeeze(2)
        pooled_2 = F.max_pool1d(conved_2, conved_2.shape[2]).squeeze(2)

        #pooled_n = [batch size, n_filters]

        cat = self.dropout(torch.cat((pooled_0, pooled_1, pooled_2), dim = 1))

        #cat = [batch size, n_filters * len(filter_sizes)]

        return self.fc(cat)

Currently the CNN model can only use 3 different sized filters, but we can actually improve the code of our model to make it more generic and take any number of filters.

We do this by placing all of our convolutional layers in a nn.ModuleList, a function used to hold a list of PyTorch nn.Modules. If we simply used a standard Python list, the modules within the list cannot be "seen" by any modules outside the list which will cause us some errors.

We can now pass an arbitrary sized list of filter sizes and the list comprehension will create a convolutional layer for each of them. Then, in the forward method we iterate through the list applying each convolutional layer to get a list of convolutional outputs, which we also feed through the max pooling in a list comprehension before concatenating together and passing through the dropout and linear layers.

## Conv2d

In [61]:
## Model
class CNNModel(pl.LightningModule):

  def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout, learning_rate):
    super().__init__()
    self.learning_rate = learning_rate

    ## define loss & metrics
    self.loss_fn = nn.BCELoss()
    self.train_accuracy = Accuracy(task = "binary", num_classes = 2, threshold= 0.5)
    self.val_accuracy = Accuracy(task = "binary", num_classes = 2, threshold= 0.5)

    ## Define Model
    self.embedding = nn.Embedding(num_embeddings= vocab_size, embedding_dim=embedding_dim, padding_idx= vocab['__PAD__'])
    self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1,
                                              out_channels = n_filters,
                                              kernel_size = (fs, embedding_dim))
                                    for fs in filter_sizes
                                    ])

    self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
    self.dropout = nn.Dropout(dropout)
    self.sigmoid = nn.Sigmoid()


  def forward(self,feature, verbose = False):
    #text = [batch size, sent len]

    embedded = self.embedding(feature)

    #embedded = [batch size, sent len, emb dim]

    embedded = embedded.unsqueeze(1)

    #embedded = [batch size, 1, sent len, emb dim]

    conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]

    #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]

    pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]

    #pooled_n = [batch size, n_filters]

    cat = self.dropout(torch.cat(pooled, dim = 1))

    #cat = [batch size, n_filters * len(filter_sizes)]

    output = self.sigmoid(self.fc(cat))
    output = output.squeeze(1)
    return output


  def _shared_step(self, batch):
    feature, label = batch['features'],batch['labels']
    logits = self(feature)
    loss = self.loss_fn(logits, label)
    return logits, loss, label

  def training_step(self, batch, batch_idx):
    logits, loss, label = self._shared_step(batch)
    self.train_accuracy(logits,label)
    self.log_dict({"train_loss": loss, "train_accuracy": self.train_accuracy}, on_step = False, on_epoch = True, prog_bar=True)
    return loss

  def validation_step(self, batch, batch_idx):
    logits, loss, label = self._shared_step(batch)
    self.val_accuracy(logits,label)
    self.log_dict({"val_loss": loss,  "val_accuracy": self.val_accuracy}, on_step = False, on_epoch = True, prog_bar = True)
    return loss

  def on_train_epoch_end(self):
    self.train_accuracy.reset()

  def on_validation_epoch_end(self):
    if self.current_epoch!=0:
      print(f"Epoch : {self.current_epoch} Validation Accuracy : {self.val_accuracy.compute()}")
    self.val_accuracy.reset()

  def configure_optimizers(self):
    optimizer = optim.Adam(self.parameters(), lr =self.learning_rate)
    return optimizer


In [62]:
## test model
model = CNNModel(vocab_size = len(vocab), embedding_dim = 100, n_filters = 100, filter_sizes = [2,3], output_dim = 1, dropout = 0.5, learning_rate = .001)
logits = model(feature,verbose = True)
print(f"Logits : {logits}")
print(f"Loss : {model.loss_fn(logits, label)}")

Logits : tensor([0.6201, 0.3390], grad_fn=<SqueezeBackward1>)
Loss : 1.024906873703003


In [63]:
## logger
logger = pl.loggers.CSVLogger("logs", name="sentiment_analysis")

## checkpoints
checkpoint_callback  = pl.callbacks.ModelCheckpoint(
                                                filename='{epoch}-{val_loss:.2f}-{val_accuracy:.2f}',
                                                every_n_epochs = 2,
                                                save_top_k = -1,
                                                monitor='val_loss',
                                                )


model2d = CNNModel(vocab_size = len(vocab),
                 embedding_dim = 100,
                 n_filters = 16,
                 filter_sizes = [1,2],
                 output_dim = 1,
                 dropout = 0.5,
                 learning_rate = .0001)

trainer = pl.Trainer(accelerator="cpu",
                     max_epochs = 5,
                     check_val_every_n_epoch=1,
                     callbacks=[checkpoint_callback],
                     logger=logger

                    )

## Train the Model
trainer.fit(model2d, train_dl, val_dl)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name           | Type           | Params
--------------------------------------------------
0 | loss_fn        | BCELoss        | 0     
1 | train_accuracy | BinaryAccuracy | 0     
2 | val_accuracy   | BinaryAccuracy | 0     
3 | embedding      | Embedding      | 909 K 
4 | convs          | ModuleList     | 4.8 K 
5 | fc             | Linear         | 33    
6 | dropout        | Dropout        | 0     
7 | sigmoid        | Sigmoid        | 0     
--------------------------------------------------
914 K     Trainable params
0         Non-trainable params
914 K     Total params
3.656     Total estimated model params size (

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Epoch : 3 Validation Accuracy : 0.9749875068664551


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch : 4 Validation Accuracy : 0.9814907312393188


## Conv1d

In [23]:
## Model
class CNN1dModel(pl.LightningModule):

  def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout, learning_rate):
    super().__init__()
    self.learning_rate = learning_rate

    ## define loss & metrics
    self.loss_fn = nn.BCELoss()
    self.train_accuracy = Accuracy(task = "binary", num_classes = 2, threshold= 0.5)
    self.val_accuracy = Accuracy(task = "binary", num_classes = 2, threshold= 0.5)

    ## Define Model
    self.embedding = nn.Embedding(num_embeddings= vocab_size, embedding_dim=embedding_dim, padding_idx= vocab['__PAD__'])
    self.convs = nn.ModuleList([
                                    nn.Conv1d(in_channels = embedding_dim,
                                              out_channels = n_filters,
                                              kernel_size = fs)
                                    for fs in filter_sizes
                                    ])

    self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
    self.dropout = nn.Dropout(dropout)
    self.sigmoid = nn.Sigmoid()


  def forward(self,feature, verbose = False):
    #feature = [batch size, sent len]

    embedded = self.embedding(feature)

    #embedded = [batch size, sent len, emb dim]

    embedded = embedded.permute(0, 2, 1)

    #embedded = [batch size, emb dim, sent len]

    conved = [F.relu(conv(embedded)) for conv in self.convs]

    #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]

    pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]

    #pooled_n = [batch size, n_filters]

    cat = self.dropout(torch.cat(pooled, dim = 1))

    #cat = [batch size, n_filters * len(filter_sizes)]

    output = self.sigmoid(self.fc(cat))
    output = output.squeeze(1)
    return output


  def _shared_step(self, batch):
    feature, label = batch['features'],batch['labels']
    logits = self(feature)
    loss = self.loss_fn(logits, label)
    return logits, loss, label

  def training_step(self, batch, batch_idx):
    logits, loss, label = self._shared_step(batch)
    self.train_accuracy(logits,label)
    self.log_dict({"train_loss": loss, "train_accuracy": self.train_accuracy}, on_step = False, on_epoch = True, prog_bar=True)
    return loss

  def validation_step(self, batch, batch_idx):
    logits, loss, label = self._shared_step(batch)
    self.val_accuracy(logits,label)
    self.log_dict({"val_loss": loss,  "val_accuracy": self.val_accuracy}, on_step = False, on_epoch = True, prog_bar = True)
    return loss

  def on_train_epoch_end(self):
    self.train_accuracy.reset()

  def on_validation_epoch_end(self):
    if self.current_epoch!=0:
      print(f"Epoch : {self.current_epoch} Validation Accuracy : {self.val_accuracy.compute()}")
    self.val_accuracy.reset()

  def configure_optimizers(self):
    optimizer = optim.Adam(self.parameters(), lr =self.learning_rate)
    return optimizer


In [43]:
## test model
model = CNN1dModel(vocab_size = len(vocab),
                   embedding_dim = 100,
                   n_filters = 100,
                   filter_sizes = [1,2],
                   output_dim = 1,
                   dropout = 0.5,
                   learning_rate = .001)

logits = model(feature,verbose = True)
print(f"Logits : {logits}")
print(f"Loss : {model.loss_fn(logits, label)}")

Logits : tensor([0.5946, 0.6049], grad_fn=<SqueezeBackward1>)
Loss : 0.7027345895767212


In [44]:
## logger
logger = pl.loggers.CSVLogger("logs", name="sentiment_analysis")

## checkpoints
checkpoint_callback  = pl.callbacks.ModelCheckpoint(
                                                filename='{epoch}-{val_loss:.2f}-{val_accuracy:.2f}',
                                                every_n_epochs = 2,
                                                save_top_k = -1,
                                                monitor='val_loss',
                                                )


model1d = CNN1dModel(vocab_size = len(vocab),
                 embedding_dim = 100,
                 n_filters = 100,
                 filter_sizes = [1,2],
                 output_dim = 1,
                 dropout = 0.5,
                 learning_rate = .001)

trainer = pl.Trainer(accelerator="cpu",
                     max_epochs = 5,
                     check_val_every_n_epoch=1,
                     callbacks=[checkpoint_callback],
                     logger=logger

                    )

## Train the Model
trainer.fit(model1d, train_dl, val_dl)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name           | Type           | Params
--------------------------------------------------
0 | loss_fn        | BCELoss        | 0     
1 | train_accuracy | BinaryAccuracy | 0     
2 | val_accuracy   | BinaryAccuracy | 0     
3 | embedding      | Embedding      | 909 K 
4 | convs          | ModuleList     | 30.2 K
5 | fc             | Linear         | 201   
6 | dropout        | Dropout        | 0     
7 | sigmoid        | Sigmoid        | 0     
--------------------------------------------------
939 K     Trainable params
0         Non-trainable params
939 K     Total params
3.758     Total estimated model params size (

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Epoch : 3 Validation Accuracy : 0.9919959902763367


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch : 4 Validation Accuracy : 0.9924962520599365


## Predict

In [74]:
def predict_sentiment(model, sentence, min_len = 2):
    model.eval()
    tokenized = process_tweet(sentence)


    if len(tokenized) < min_len:
        tokenized += ['__PAD__'] * (min_len - len(tokenized))

    # print(tokenized)
    indexed = tweet_to_tensor(tokenized)
    tensor = torch.LongTensor(indexed)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [79]:
predict_sentiment(model2d , "this movie is best :)")

0.7244307398796082